## Animando os vértice do triângulo

Neste exemplo, é utilizada a mesma abordagem de animação do notebook ([20_Animando_as_cores_dos_triangulos](20_Animando_as_cores_dos_triangulos.ipynb)). A diferença está no dado que foi animado. Neste exemplo, foi animada a posição de um dos vértices de cada triângulo. Para isso, foi utilizada a variável 'interpolation_factor' para calcular as novas posições de vértices ('triangle01_new_vertex_v3' e 'triangle02_new_vertex_v1'), um para cada triângulo. Os vetores de posição de cada triângulo (vertexPosition01 e vertexPosition02) são atualizados com as novas posições e depois esses vetores são utilizados para atualizar os dados que estão na GPU através do método 'updateVertexPositions()'.

In [1]:
import time
import math
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v1 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v1 import ModelRenderer

class MyWidget(QtOpenGL.QGLWidget):
    
    def initializeGL(self):
        
        # posição de cada vértice do primeiro triângulos
        self.vertexPosition01 = np.array([
            -0.90, -0.90, 0.0, 1.0, # Triângulo 1
             0.85, -0.90, 0.0, 1.0,
            -0.90,  0.85, 0.0, 1.0],
            dtype=np.float32)
        
        # posição de cada vértice do segundo triângulos
        self.vertexPosition02 = np.array([
             0.90, -0.85, 0.0, 1.0, # Triângulo 2
             0.90,  0.90, 0.0, 1.0,
            -0.85,  0.90, 0.0, 1.0],
            dtype=np.float32)
        
        # cria para cada triângulo um objeto responsável por carregar os dados para a GPU e renderizá-los
        self.triangleRenderer01 = ModelRenderer(self.vertexPosition01)
        self.triangleRenderer02 = ModelRenderer(self.vertexPosition02)
        
        # cria um shader program simples
        self.shaderProgram = SimpleShaderProgram()
        
        # ativa o shader programa para permitir configurar uma cor única para todos os vértices
        self.shaderProgram.bind()
        self.shaderProgram.useUniformColor(True)
        self.shaderProgram.setUniformColor(np.array([0.0, 0.0, 0.5, 1.0], dtype=np.float32))
        self.shaderProgram.release()

        # recupera o endereços da variável de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        
        # configura os dados do modelo para serem os dados de entrada do shader program
        self.triangleRenderer01.setVertexPositionLoc(position_loc)
        self.triangleRenderer02.setVertexPositionLoc(position_loc)

        # armazena o momento que o programa começou
        self.startTime = time.time()
        
    def paintGL(self):
        
        # configura a cor de background
        gl.glClearColor(0, 0, 0, 1)
        
        # limpa o background com a cor especificada
        gl.glClear(gl.GL_COLOR_BUFFER_BIT)
        
        # calcula o tempo de execução do programa
        self.currentTime = time.time()
        time_difference = self.currentTime - self.startTime
        
        # calcula o fator de interpolação
        interpolation_factor = (math.sin(time_difference) + 1) / 2
        
        triangle01_v1 = np.array([-0.90, -0.90, 0.0, 1.0], dtype=np.float32)
        triangle01_v3 = np.array([-0.90,  0.85, 0.0, 1.0], dtype=np.float32)
        
        triangle02_v1 = np.array([0.90, -0.85, 0.0, 1.0], dtype=np.float32)
        triangle02_v2 = np.array([0.90,  0.90, 0.0, 1.0], dtype=np.float32)
        
        # calcula o novo vértice de cada triângulo
        triangle01_new_vertex_v3 = interpolation_factor * triangle01_v3 + (1 - interpolation_factor) * triangle01_v1
        triangle02_new_vertex_v1 = interpolation_factor * triangle02_v1 + (1 - interpolation_factor) * triangle02_v2
        
        # atualiza os dados nos vetores
        self.vertexPosition01[8:] = triangle01_new_vertex_v3
        self.vertexPosition02[0:4] = triangle02_new_vertex_v1
        
        # atualiza os dados dos triângulso na GPU
        self.triangleRenderer01.updateVertexPositions(self.vertexPosition01)
        self.triangleRenderer02.updateVertexPositions(self.vertexPosition02)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # renderiza o primeiro triângulo
        self.triangleRenderer01.render()
        
        # renderiza o segundo triângulo
        self.triangleRenderer02.render()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()

    def resizeGL(self, width, height):
        
        # atualiza a área de renderização para ser a janela inteira
        gl.glViewport(0, 0, width, height)

def main():
    import sys

    #Criação de um aplicativo Qt
    app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(640, 480)
    w.setWindowTitle('OpenGL example')
    w.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
! jupyter nbconvert --to python 21_Animando_os_vertice_do_triangulo.ipynb
%run -i 21_Animando_os_vertice_do_triangulo.py

[NbConvertApp] Converting notebook 21_Animando_as_posicoes_do_vertice_do_triangulo.ipynb to python
[NbConvertApp] Writing 5561 bytes to 21_Animando_as_posicoes_do_vertice_do_triangulo.py
